In [9]:
import tensorflow as tf
import json
import os
import logging  # To log errors or missing files

## Images rename to image_id

In [20]:
os.getcwd()

'D:\\Projects\\DL\\MonumentDetection\\DL\\keras_cv'

In [27]:
os.chdir('coco-nyaptola')

In [31]:
os.getcwd()

'D:\\Projects\\DL\\MonumentDetection\\DL\\keras_cv\\coco-nyaptola'

In [35]:
count = 0
image_dir = 'images'
for file in os.listdir(image_dir):
    old_filename = os.path.join(image_dir, file)
    new_filename = os.path.join(image_dir, str(count) + '.png')
    os.rename(old_filename, new_filename)
    count += 1

## Keras_CV Compatible Annotation file

In [36]:
def load_and_convert_annotations(annotations_file, output_file):
  with open(annotations_file, 'r') as f:
    annotations = json.load(f)

  xywh_annotations = convert_coco_to_xywh(annotations)

  with open(output_file, 'w') as f:
    json.dump(xywh_annotations, f, indent=2) 

In [37]:
def convert_coco_to_xywh(annotations):
  #Converts bounding boxes in COCO format to xywh format.
  xywh_annotations = []
  for ann in annotations["annotations"]:
    image_id = ann["image_id"]
    bbox = ann["bbox"]  # Assuming bbox is in [xmin, ymin, width, height] format
    x_min, y_min, width, height = bbox

    # Convert to xywh format
    x_center = x_min + width / 2
    y_center = y_min + height / 2

    xywh_annotations.append({
      "image_id": image_id,
      "bbox": [x_center, y_center, width, height],
      "category_id": ann["category_id"],  # Assuming category_id exists
    })
  return xywh_annotations

In [40]:
annotation_file = "result.json"
annotation_file = os.path.abspath(annotation_file)
output_file = "annotations.json"

In [41]:
converted_annotations = load_and_convert_annotations(annotation_file,output_file)

## Conversion of annotations to yolo format

In [2]:
!pip install xmltodict


DEPRECATION: Loading egg at c:\users\acer\anaconda3\envs\object_detection\lib\site-packages\avro_python3-1.10.2-py3.9.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation.. Discussion can be found at https://github.com/pypa/pip/issues/12330


In [4]:
import xmltodict
import os

# Function to convert XML to YOLO format
def convert_annotation(xml_file):
    with open(xml_file, 'r') as file:
        annotation = xmltodict.parse(file.read())['annotation']
    
    yolo_annotations = []
    
    # Check if 'object' is a list or a single item
    objects = annotation.get('object', [])
    
    if isinstance(objects, dict):
        # If it's a dictionary, convert it to a list for consistent processing
        objects = [objects]

    for obj in objects:
        class_name = obj['name']
        bbox = obj['bndbox']
        
        xmin = int(bbox['xmin'])
        ymin = int(bbox['ymin'])
        xmax = int(bbox['xmax'])
        ymax = int(bbox['xmax'])
        
        # Calculate YOLO format
        image_width = int(annotation['size']['width'])
        image_height = int(annotation['size']['height'])

        x_center = (xmin + xmax) / 2 / image_width
        y_center = (ymin + ymax) / 2 / image_height
        width = (xmax - xmin) / image_width
        height = (ymax - ymin) / image_height
        
        class_id = 0  # Adjust according to your class mapping
        yolo_annotations.append(f"{class_id} {x_center:.6f} {y_center:.6f} {width:.6f} {height:.6f}")

    return yolo_annotations


In [5]:
# Loop over all XML annotation files
for annotation_file in os.listdir(annotations_path):
    if annotation_file.endswith('.xml'):
        xml_path = os.path.join(annotations_path, annotation_file)
        
        # Convert XML to YOLO format
        yolo_annotations = convert_annotation(xml_path)
        
        # Create corresponding text file for YOLO
        txt_filename = annotation_file.replace('.xml', '.txt')
        txt_path = os.path.join(output_path, txt_filename)
        
        with open(txt_path, 'w') as txt_file:
            txt_file.write('\n'.join(yolo_annotations))

In [6]:
import os
import shutil
import random

# Set paths
images_path = "dataset/images/"
yolo_labels_path = "dataset/yolo_labels/"

# Create directories for training and validation
os.makedirs("dataset/train/images", exist_ok=True)
os.makedirs("dataset/train/labels", exist_ok=True)
os.makedirs("dataset/val/images", exist_ok=True)
os.makedirs("dataset/val/labels", exist_ok=True)

# List all images
images = os.listdir(images_path)

# Shuffle images and split into training and validation
random.shuffle(images)
train_ratio = 0.8
num_train = int(len(images) * train_ratio)

train_images = images[:num_train]
val_images = images[num_train:]

# Move images and corresponding labels to respective directories
for image in train_images:
    base_name = os.path.splitext(image)[0]
    label_file = f"{base_name}.txt"
    
    shutil.copy(os.path.join(images_path, image), "dataset/train/images/")
    shutil.copy(os.path.join(yolo_labels_path, label_file), "dataset/train/labels/")

for image in val_images:
    base_name = os.path.splitext(image)[0]
    label_file = f"{base_name}.txt"
    
    shutil.copy(os.path.join(images_path, image), "dataset/val/images/")
    shutil.copy(os.path.join(yolo_labels_path, label_file), "dataset/val/labels/")
